# ch14. python connect
이제 python에서 RDBMS를 사용하는 방법을 알아보겠습니다. 먼저 python에서 RDMBS를 사용하기 위한 패키지를 설치해야합니다. 종류는 여러가지가 있지만, 가장 대표적인 pymysql을 사용해보겠습니다. 이런 종류의 라이브러리들을 보통 DB driver라고 부릅니다.

## pymysql을 이용해서 RDBMS 접속하기

In [4]:
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.2 MB/s eta 0:00:00


이제 pymysql로 DB에 접속해보겠습니다. 접속을 관리하는 conn (connection의 약자입니다. 보통 이렇게 줄여서 많이 사용합니다.) 객체를 만들어서 사용합니다.

In [1]:
import pymysql 

conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="mystore"
)

In [2]:
conn, type(conn)

(<pymysql.connections.Connection at 0x1069bd480>,
 pymysql.connections.Connection)

pymysql 라이브러리 안에 정의되어 있는 Connection 클래스로 만든 객체라는 소리입니다. 

## Read 쿼리 날리기
이제 이 conn으로 쿼리를 날려보겠습니다. 쿼리를 날리기 위해서는 Connection 클래스 안에 정의되어 있는 cursor를 사용해야 합니다.

쿼리를 날리기 위해서는 먼저 SQL 문을 문자열로 만들어야 합니다. 그 다음, 마치 파일을 열고 데이터를 읽어오듯 cursor 객체를 만든 뒤에 execute 함수에 sql문을 파라미터로 전달해서 쿼리를 날리면 됩니다. 그 다음, fetchall을 이용해서 결과를 읽어오면 됩니다.

In [3]:
def read_all_items():
    select_sql = "SELECT * FROM products"
    with conn.cursor() as cursor:
        cursor.execute(select_sql)
        result = cursor.fetchall()
    return result

In [4]:
records = read_all_items()
print(records)

((1, '검은색 모자', 3000, '야외 활동에 적합한 스포티한 모자'), (2, '와이드 청바지', 20000, '활동성을 고려한 편안한 청바지'), (3, '흰색 컨버스화', 5000, '편안하면서도 예쁜 운동화'), (4, '흰색 무지티', 5000, '가성비 좋은 흰색 무지티'), (5, '검은색 모자', 50000, '고급 브랜드의 야심찬 신상 모자'), (6, '노란색 원피스', 68000, '하늘하늘한 원피스'), (7, '하늘색 비니', 12200, '따뜻하고 산뜻한 비니'))


### 연습 문제
- titanic 테이블에서 Pclass, Survived, Name, Age 컬럼을 Age 오름차순으로 10개만 읽어와서 print 문으로 출력해보세요.

## Write 쿼리 날리기
write 쿼리는 read 쿼리와 다르게 execute로 쿼리를 실행한 다음, commit을 해주어야 결과가 반영이 됩니다. 이를 온전히 이해하기 위해서는 transaction 개념을 이해해야 하는데, 일단은 넘어가고 데이터를 보호하기 위한 안전장치라고 생각하고 넘어갑시다.

products 테이블에 새로운 레코드를 하나 추가해보겠습니다. 이 때, python for문을 사용해서 편하게 SQL문을 작성해보겠습니다.

In [5]:
new_products = [
    ("노란색 원피스", 68000, "하늘하늘한 원피스"),
    ("하늘색 비니", 12200, "따뜻하고 산뜻한 비니"),
]

In [6]:
def generate_insert_sql(new_products):
    insert_sql = "INSERT INTO products(name, price, description) VALUES "
    for name, price, description in new_products:
        insert_sql += f"('{name}', {price}, '{description}'), "
    # 마지막에 붙은 , 떼주기
    insert_sql = insert_sql[:-2]
    return insert_sql

In [7]:
insert_sql = generate_insert_sql(new_products)
insert_sql

"INSERT INTO products(name, price, description) VALUES ('노란색 원피스', 68000, '하늘하늘한 원피스'), ('하늘색 비니', 12200, '따뜻하고 산뜻한 비니')"

In [8]:
with conn.cursor() as cursor:
    cursor.execute(insert_sql)
    conn.commit()

In [9]:
read_all_items()

((1, '검은색 모자', 3000, '야외 활동에 적합한 스포티한 모자'),
 (2, '와이드 청바지', 20000, '활동성을 고려한 편안한 청바지'),
 (3, '흰색 컨버스화', 5000, '편안하면서도 예쁜 운동화'),
 (4, '흰색 무지티', 5000, '가성비 좋은 흰색 무지티'),
 (5, '검은색 모자', 50000, '고급 브랜드의 야심찬 신상 모자'),
 (6, '노란색 원피스', 68000, '하늘하늘한 원피스'),
 (7, '하늘색 비니', 12200, '따뜻하고 산뜻한 비니'),
 (10, '노란색 원피스', 68000, '하늘하늘한 원피스'),
 (11, '하늘색 비니', 12200, '따뜻하고 산뜻한 비니'))

## 정리

이번 챕터에서는 python 코드 상에서 RDBMS에 어떻게 접속하고, SQL문을 날릴 수 있는지 알아보았습니다. 앞으로 자주 사용하게 될 코드들이니 사용법을 잘 익혀놓으시기 바랍니다.